<h1 style="text-align: center;">Predicción del precio de los vuelos</h1>

# Importamos las librerias

In [1]:
library(MASS)
library(leaps)
library(alr4)
library(car)

Loading required package: car

Loading required package: carData

Loading required package: effects

lattice theme set by effectsTheme()
See ?effectsTheme for details.



# Cargamos el conjunto de datos

In [2]:
datos_3 <- read.csv("C:/Users/marti/OneDrive/Escritorio/jupyter/database/Clean_Dataset.csv")

head(datos_3)

,X,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<int>
1,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
2,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
3,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
4,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
5,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
6,5,Vistara,UK-945,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.33,1,5955


### Eliminamos la columna X, ya que es una variable de índices que no agrega información relevante a nuestros datos

In [3]:
datos_3<-subset(datos_3,select=-X)

## Convirtiendo en factores las columnas de cadenas.

In [4]:
datos_3$airline <-as.factor(datos_3$airline)
datos_3$source_city <-as.factor(datos_3$source_city)
datos_3$departure_time <-as.factor(datos_3$departure_time)
datos_3$arrival_time <-as.factor(datos_3$arrival_time)
datos_3$destination_city<-as.factor(datos_3$destination_city)
datos_3$class <-as.factor(datos_3$class)
datos_3$stops <-as.factor(datos_3$stops)
datos_3$flight <-as.factor(datos_3$flight)

## Transformamos la columna 'days_left' en categórica, donde una categoría corresponde a los valores 1 o 2, y la otra categoría a los valores mayores a 2.

In [5]:
datos_3$days_left <- cut(datos_3$days_left, breaks = c(-Inf, 2, Inf), labels = c("1 o 2", "Mayor a 2"), include.lowest = TRUE)

## Visualizamos los cambios realizados anteriormente.

In [6]:
head(datos_3)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<int>
1,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1 o 2,5953
2,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1 o 2,5953
3,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1 o 2,5956
4,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1 o 2,5955
5,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1 o 2,5955
6,Vistara,UK-945,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.33,1 o 2,5955


## Resumen descriptivo de nuestros datos

In [7]:
summary(datos_3)

      airline           flight          source_city          departure_time 
 Air_India: 80892   UK-706 :  3235   Bangalore:52061   Afternoon    :47794  
 AirAsia  : 16098   UK-772 :  2741   Chennai  :38700   Early_Morning:66790  
 GO_FIRST : 23173   UK-720 :  2650   Delhi    :61343   Evening      :65102  
 Indigo   : 43120   UK-836 :  2542   Hyderabad:40806   Late_Night   : 1306  
 SpiceJet :  9011   UK-822 :  2468   Kolkata  :46347   Morning      :71146  
 Vistara  :127859   UK-828 :  2440   Mumbai   :60896   Night        :48015  
                    (Other):284077                                          
         stops               arrival_time    destination_city      class       
 one        :250863   Afternoon    :38139   Bangalore:51068   Business: 93487  
 two_or_more: 13286   Early_Morning:15417   Chennai  :40368   Economy :206666  
 zero       : 36004   Evening      :78323   Delhi    :57360                    
                      Late_Night   :14001   Hyderabad:42726     

# ¿El precio varía según las aerolíneas?

## Ajustamos el modelo de regresion lineal

In [8]:
a <- lm(price ~ airline,data=datos_3)
summary(a)


Call:
lm(formula = price ~ airline, data = datos_3)

Residuals:
   Min     1Q Median     3Q    Max 
-28683 -18364  -1365  15352  92674 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      23507.02      70.36  334.08   <2e-16 ***
airlineAirAsia  -19415.95     172.71 -112.42   <2e-16 ***
airlineGO_FIRST -17855.01     149.11 -119.75   <2e-16 ***
airlineIndigo   -18182.80     119.33 -152.38   <2e-16 ***
airlineSpiceJet -17327.74     222.25  -77.97   <2e-16 ***
airlineVistara    6889.52      89.91   76.63   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 20010 on 300147 degrees of freedom
Multiple R-squared:  0.2227,	Adjusted R-squared:  0.2226 
F-statistic: 1.719e+04 on 5 and 300147 DF,  p-value: < 2.2e-16


$$E(\text{Air_India})=23507 $$
$$E(\text{AirAsiaT})=23507-19415=4092$$
$$E(\text{GO_FIRST})=23507 -17855=5652$$
$$E(\text{Indigo})=23507 -18182= 5325$$
$$E(\text{SpiceJet})=23507 -17327=6180$$
$$E(\text{Vistara})=23507 +6889= 30396$$

- en el precio si importa de que aereolina se compra el boleto

# ¿Cómo se ve afectado el precio cuando los billetes se compran sólo 1 o 2 días antes de la salida?

## Ajustando el modelo de regresion

In [9]:
b <- lm(price ~ days_left, data=datos_3)
summary(b)


Call:
lm(formula = price ~ days_left, data = datos_3)

Residuals:
   Min     1Q Median     3Q    Max 
-25444 -16002 -13370  21764 102314 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)         27421.2      293.9   93.29   <2e-16 ***
days_leftMayor a 2  -6663.7      296.9  -22.45   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 22680 on 300151 degrees of freedom
Multiple R-squared:  0.001676,	Adjusted R-squared:  0.001672 
F-statistic: 503.8 on 1 and 300151 DF,  p-value: < 2.2e-16


$$E(\text{1 o 2})=27421 $$
$$E(\text{Mayor a 2}) =27421-6663=20758$$

# ¿El precio del billete cambia según la hora de salida y la hora de llegada?

In [10]:
c <- lm(price ~ departure_time+arrival_time, data=datos_3)
summary(c)


Call:
lm(formula = price ~ departure_time + arrival_time, data = datos_3)

Residuals:
   Min     1Q Median     3Q    Max 
-23133 -16337 -11769  21377 100974 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  15564.4      154.1 101.004   <2e-16 ***
departure_timeEarly_Morning   2148.4      136.2  15.773   <2e-16 ***
departure_timeEvening         3747.9      136.8  27.401   <2e-16 ***
departure_timeLate_Night     -5650.2      634.0  -8.912   <2e-16 ***
departure_timeMorning         3101.3      133.6  23.210   <2e-16 ***
departure_timeNight           5580.9      148.1  37.688   <2e-16 ***
arrival_timeEarly_Morning    -3817.4      216.7 -17.613   <2e-16 ***
arrival_timeEvening           4729.2      140.6  33.641   <2e-16 ***
arrival_timeLate_Night       -7352.7      225.3 -32.634   <2e-16 ***
arrival_timeMorning           3299.8      147.5  22.377   <2e-16 ***
arrival_timeNight             3431.1      138.1  24.845   <2e-16 **

# ¿Cómo cambia el precio con el cambio en Origen y Destino?

In [12]:
c<-lm(price ~ source_city+destination_city,data=datos_3)
summary(c)


Call:
lm(formula = price ~ source_city + destination_city, data = datos_3)

Residuals:
   Min     1Q Median     3Q    Max 
-21899 -16129 -12526  22434 104269 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)                22835.9      150.1 152.140  < 2e-16 ***
source_cityChennai           506.5      153.4   3.302 0.000959 ***
source_cityDelhi           -3681.3      139.4 -26.409  < 2e-16 ***
source_cityHyderabad       -1848.7      152.2 -12.149  < 2e-16 ***
source_cityKolkata           162.8      147.5   1.104 0.269735    
source_cityMumbai           -171.4      138.9  -1.234 0.217131    
destination_cityChennai      409.6      152.2   2.691 0.007122 ** 
destination_cityDelhi      -4196.9      142.3 -29.502  < 2e-16 ***
destination_cityHyderabad  -1664.9      150.9 -11.033  < 2e-16 ***
destination_cityKolkata      280.9      145.6   1.929 0.053705 .  
destination_cityMumbai      -307.8      140.5  -2.190 0.028542 *  
---
Signif. codes:  0

# Ajustando el modelo para predicir el precio 

In [14]:
f<- lm(price ~ airline+source_city+departure_time+stops+arrival_time+destination_city+class+duration+days_left, data=datos_3)
summary(f)


Call:
lm(formula = price ~ airline + source_city + departure_time + 
    stops + arrival_time + destination_city + class + duration + 
    days_left, data = datos_3)

Residuals:
   Min     1Q Median     3Q    Max 
-38792  -3372   -632   3436  67369 

Coefficients:
                              Estimate Std. Error   t value Pr(>|t|)    
(Intercept)                  57314.086    120.852   474.249  < 2e-16 ***
airlineAirAsia                -285.051     64.205    -4.440 9.01e-06 ***
airlineGO_FIRST               1500.930     55.650    26.971  < 2e-16 ***
airlineIndigo                 1927.808     48.217    39.982  < 2e-16 ***
airlineSpiceJet               2387.604     78.580    30.385  < 2e-16 ***
airlineVistara                3953.716     31.730   124.604  < 2e-16 ***
source_cityChennai             -32.338     47.206    -0.685    0.493    
source_cityDelhi             -1415.698     42.851   -33.037  < 2e-16 ***
source_cityHyderabad         -1669.261     46.837   -35.640  < 2e-16 ***
sour

- Nuestro modelo tiene casi todas sus variables regresoras son categoricas  exepto la variable regresora duration.
- Esto da como resultado muchos modelos de la forma 
$$y=\beta_x+\beta_1 x$$
donde solo cambia el intercepto, dependiendo de lo que se escoge en las variables categoricas. $x=duration$.

$$0 <duration < 50$$

$$ 6.83 <duration<16.67 $$